### Prérecquis : 
- capsule *Configuration du robot*
- capsule *Prise en main d'un notebook Jupyter*
- capsule *Python : les bases*
- capsule *Python : les tableaux ndarrays de numpy*
- capsule *Python : tracé de courbes avec matplotlib*
- capsule *Python Orienté Objet*

### Acquis d'aprentissage visés : 
A l'issue de cette activité l'apprenant saura : 
- utiliser un notebook Jupyter pour faire bouger le robot 
- faire bouger le robot en utilisant TrajectoryRecorder

## Enregistrer un mouvement et le reproduire


In [1]:
import time 

from reachy_sdk import ReachySDK
from reachy_sdk.trajectory import goto

reachy = ReachySDK('localhost')

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses"
	debug_error_string = "{"created":"@1623921188.413056627","description":"Failed to pick subchannel","file":"src/core/ext/filters/client_channel/client_channel.cc","file_line":3008,"referenced_errors":[{"created":"@1623921188.413055639","description":"failed to connect to all addresses","file":"src/core/ext/filters/client_channel/lb_policy/pick_first/pick_first.cc","file_line":397,"grpc_status":14}]}"
>

Si l'on veut enregistrer les mouvements du bras : 

In [2]:
recorded_joints = [
    reachy.r_arm.r_shoulder_pitch,
    reachy.r_arm.r_shoulder_roll,
    reachy.r_arm.r_arm_yaw,
    reachy.r_arm.r_elbow_pitch,
    reachy.r_arm.r_forearm_yaw,
    reachy.r_arm.r_wrist_pitch,
    reachy.r_arm.r_wrist_roll,
]

sampling_frequency = 100  #en hertz
record_duration = 10  #en seconde

Le programme va enregistrer les déplacements du robot pendant 10 secondes. Fait attention a ce que le robot se trouve dans un endroit où il peut bouger librement sans objets ou obstacles autour de lui.


In [3]:
reachy.turn_off('r_arm')

In [10]:
trajectories = [] #on créer une nouvelle liste trajectoire

start = time.time() #seconde passées depuis epoch
while (time.time() - start) < record_duration:
    #on optient les positions actuelles de toutes les jointures 
    current_point = [joint.present_position for joint in recorded_joints]
    #on ajoute les positions à la liste trajectoire 
    trajectories.append(current_point)

    time.sleep(1 / sampling_frequency)

In [13]:
trajectories

[[11.01, -10.44, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.53, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.53, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.44, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.53, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.53, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.53, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.53, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.53, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.53, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.53, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.53, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.53, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.53, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.53, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.53, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.53, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.53, 12.88, -85.67, -8.06, -3.3, -6.01],
 [11.01, -10.53, 12.88, -85.67, -8.06, -3.3, -

In [9]:
reachy.r_arm

<Arm side="right" joints=<Holder
	<Joint name="r_shoulder_pitch" pos="11.01" mode="compliant">
	<Joint name="r_shoulder_roll" pos="-10.53" mode="compliant">
	<Joint name="r_arm_yaw" pos="12.88" mode="compliant">
	<Joint name="r_elbow_pitch" pos="-85.67" mode="compliant">
	<Joint name="r_forearm_yaw" pos="-8.06" mode="compliant">
	<Joint name="r_wrist_pitch" pos="-3.30" mode="compliant">
	<Joint name="r_wrist_roll" pos="-6.01" mode="compliant">
	<Joint name="r_gripper" pos="-9.60" mode="compliant">
>>

On met le robot en mode rigide

In [5]:
reachy.turn_on('r_arm')

On lance l'enregistrement du bras 

In [6]:
#on rend rigide toutes jointures utiliser pour effectuer la trajectoire
for joint in recorded_joints:
    joint.compliant = False

#on créer un dictionnaire associant chaque jointure à sa première position 
first_point = dict(zip(recorded_joints, trajectories[0]))

#le robot se positionne sur les premieres positions de chaque jointures 
goto(first_point, duration=3.0)

In [7]:
for joints_positions in trajectories:
    for joint, pos in zip(recorded_joints, joints_positions):
        joint.goal_position = pos

    time.sleep(1 / sampling_frequency)

In [8]:
reachy.turn_off_smoothly('r_arm')